# Model Validation and Data Leakage

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, make_scorer

#from useful_functions import *

%load_ext autoreload
%autoreload 2

## Objectives

- explain the bias-variance tradeoff and the correlative notions of underfit and overfit models
- describe a train-test split and explain its purpose in the context of predictive statistics / machine learning
- explain the algorithm of cross-validation
- use best practices for building non-leaky workflows
- repair leaky workflows

## Motivation

At this point, we have seen different ways to create models from our data through different linear regression techniques. That's good. But when it comes to measuring model performance, we also want to make sure that our models are ready to predict on data that they haven't seen yet.

Usually, when our model is ready to be used in the "real world" we refer to this as putting our model into **production** or **deploying** our model. The data points for which it will make predictions will be data *it has never seen before*, as opposed to the data points that were used to train the model.

This is where ***model validation*** techniques come in, namely, to ensure our model can *generalize* to data it hasn't directly seen before.

As a way into a discussion of these techniques let's say a word about the **bias-variance tradeoff**.

## The Bias-Variance Tradeoff

We can break up how the model makes mistakes (the error) by saying there are three parts:

- Error inherent in the data (noise): **irreducible error**
- Error from not capturing signal (too simple): **bias**
- Error from "modeling noise", i.e. capturing patterns in the data that don't generalize well (too complex): **variance**

We can summarize this in an equation for the _mean squared error_ (MSE):

$MSE = Bias(\hat{y})^2 + Var(\hat{y}) + \sigma^2$

![optimal](images/optimal_bias_variance.png)
http://scott.fortmann-roe.com/docs/BiasVariance.html

### Bias

**High-bias** algorithms tend to be less complex, with simple or rigid underlying structure.

![](images/noisy-sine-linear.png)

+ They train models that are consistent, but inaccurate on average.
+ These include linear or parametric algorithms such as regression and naive Bayes.
+ The following sorts of difficulties could lead to high bias:
  - We did not include the correct predictors
  - We did not take interactions into account
  - We missed a non-linear (polynomial) relationship

      
High-bias models are generally **underfit**: The models have not picked up enough of the signal in the data. And so even though they may be consistent, they don't perform particularly well on the initial data, and so they will be consistently inaccurate.

### Variance

On the other hand, **high-variance** algorithms tend to be more complex, with flexible underlying structure.

![](images/noisy-sine-decision-tree.png)

+ They train models that are accurate on average, but inconsistent.
+ These include non-linear or non-parametric algorithms such as decision trees and nearest-neighbor models.
+ The following sorts of difficulties could lead to high variance:
  - We included an unreasonably large number of predictors;
  - We created new features by squaring and cubing each feature.

High variance models are **overfit**: The models have picked up on the noise as well as the signal in the data. And so even though they may perform well on the initial data, they will be inconsistently accurate on new data.

![](images/target.png)

### Balancing Bias and Variance

While we build our models, we have to keep this relationship in mind.  If we build complex models, we risk overfitting our models.  Their predictions will vary greatly when introduced to new data.  If our models are too simple, the predictions as a whole will be inaccurate.   

![](images/noisy-sine-third-order-polynomial.png)

The goal is to build a model with enough complexity to be accurate, but not too much complexity to be erratic.

## 🧠 Knowledge Check

![which_model](images/which_model_is_better_2.png)

## Train-Test Split

It is hard to know if your model is too simple or complex by just using it on training data.

We can _hold out_ part of our training sample, use it as a test sample, and then use it to monitor our prediction error.

This allows us to evaluate whether our model has the right balance of bias/variance. 

<img src='images/testtrainsplit.png' width =550 />

* **training set** —a subset to train a model.
* **test set**—a subset to test the trained model.

## Is the Model Overfitting or Underfitting?

If our model is not performing well on the training  data, we are probably underfitting it.  

To know if our  model is overfitting the data, we need  to test our model on unseen data. 
We then measure our performance on the unseen data. 

If the model performs significantly worse on the  unseen data, it is probably  overfitting the data.

<img src='https://developers.google.com/machine-learning/crash-course/images/WorkflowWithTestSet.svg' width=500/>

## Practice Exercises: Name that Model!

Consider the following scenarios and describe them according to bias and variance. There are four possibilities:

- a. The model has low bias and high variance.
- b. The model has high bias and low variance.
- c. The model has both low bias and low variance.
- d. The model has both high bias and high variance.

**Scenario 1**: The model has a low RMSE on training and a low RMSE on test.
<details>
    <summary> Answer
    </summary>
    c. The model has both low bias and low variance.
    </details>

**Scenario 2**: The model has a high $R^2$ on the training set, but a low $R^2$ on the test.
<details>
    <summary> Answer
    </summary>
    a. The model has low bias and high variance.
    </details>

**Scenario 3**: The model performs well on data it is fit on and well on data it has not seen.
<details>
    <summary> Answer
    </summary>
    c. The model has both low bias and low variance.
    </details>
  

**Scenario 4**: The model has a low $R^2$ on training but high on the test set.
<details>
    <summary> Answer
    </summary>
    d. The model has both high bias and high variance.
    </details>

**Scenario 5**: The model leaves out many of the meaningful predictors, but is consistent across samples.
<details>
    <summary> Answer
    </summary>
    b. The model has high bias and low variance.
    </details>

**Scenario 6**: The model is highly sensitive to random noise in the training set.
<details>
    <summary> Answer
    </summary>
    a. The model has low bias and high variance.
    </details>

## Should You Ever Fit on Your Test Set?  

![no](https://media.giphy.com/media/d10dMmzqCYqQ0/giphy.gif)

**Never fit on test data.** If you are seeing surprisingly good results on your evaluation metrics, it might be a sign that you are accidentally training on the test set.

## Train-Test Split Example

In [3]:
data = load_diabetes()

print(data.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [4]:
df = pd.concat([pd.DataFrame(data.data, columns=data.feature_names),
               pd.Series(data.target, name='target')], axis=1)

df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [5]:
df.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,442.000000
mean,-2.511817e-19,1.230790e-17,-2.245564e-16,-4.797570e-17,-1.381499e-17,3.918434e-17,-5.777179e-18,-9.042540e-18,9.293722e-17,1.130318e-17,152.133484
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,77.093005
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123988e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260971e-01,-1.377672e-01,25.000000
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665608e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324559e-02,-3.317903e-02,87.000000
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670422e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947171e-03,-1.077698e-03,140.500000
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564379e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243232e-02,2.791705e-02,211.500000
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320436e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335973e-01,1.356118e-01,346.000000


In [6]:
df.corr()['target'].map(abs).sort_values(ascending=False)

target    1.000000
bmi       0.586450
s5        0.565883
bp        0.441482
s4        0.430453
s3        0.394789
s6        0.382483
s1        0.212022
age       0.187889
s2        0.174054
sex       0.043062
Name: target, dtype: float64

In [7]:
X = df[['bmi', 's5']]
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [8]:
display(X_train.head())
display(X_test.head())

,bmi,s5
16,0.042296,0.052277
408,-0.050396,0.058038
432,0.055229,0.055686
316,0.014272,0.074966
3,-0.011595,0.022688


,bmi,s5
287,-0.006206,0.032432
211,0.036907,-0.022517
72,-0.004050,0.084492
321,0.051996,0.098648
73,-0.020218,-0.005142


In [9]:
y_train

16     166.0
408    189.0
432    173.0
316    220.0
3      206.0
       ...  
106    134.0
270    202.0
348    148.0
435     64.0
102    302.0
Name: target, Length: 331, dtype: float64

In [10]:
print(X_train.shape)
print(X_test.shape)

(331, 2)
(111, 2)


In [11]:
print(X_train.shape[0] == y_train.shape[0])
print(X_test.shape[0] == y_test.shape[0])

True
True


In [12]:
# Instanstiate your linear regression object
lr = LinearRegression()

In [13]:
# fit the model on the training set
lr.fit(X_train, y_train)

LinearRegression()

In [14]:
# Check the R^2 of the training data
lr.score(X_train, y_train)

0.43965589630857904

In [15]:
lr.coef_

array([725.32287185, 544.67500269])

A .440 R-squared reflects a model that explains about half of the total variance in the data. 

## Now check performance on test data

Next, we test how well the model performs on the unseen test data. Remember, we do not fit the model again. The model has calculated the optimal parameters learning from the training set.  

In [16]:
lr.score(X_test, y_test)

0.5112943322988956

## 🧠 Knowledge Check

How would you describe the bias of the model based on the above training $R^2$?

<details>
    <summary> Answer
    </summary>
    The difference between the train and test scores is low. High bias
    </details>

What does that indicate about variance? somewhat low variance

## Same Procedure with a Polynomial Model

In [17]:
poly_2 = PolynomialFeatures(4)

X_poly = pd.DataFrame(
            poly_2.fit_transform(df.drop('target', axis=1))
                      )

y = df.target

In [18]:
X_poly

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,1.0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,...,2.092457e-09,-2.045164e-08,1.812847e-08,-1.606919e-08,1.424384e-08,1.570600e-07,-1.392190e-07,1.234047e-07,-1.093867e-07,9.696107e-08
1,1.0,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,...,1.326016e-05,1.260051e-05,1.700266e-05,2.294276e-05,3.095810e-05,2.180143e-05,2.941804e-05,3.969561e-05,5.356378e-05,7.227698e-05
2,1.0,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,...,4.518291e-09,-6.072581e-11,5.503218e-10,-4.987238e-09,4.519636e-08,6.702846e-11,-6.074389e-10,5.504856e-09,-4.988723e-08,4.520982e-07
3,1.0,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,...,1.031672e-07,4.006640e-07,-1.653307e-07,6.822235e-08,-2.815139e-08,2.649509e-07,-1.093298e-07,4.511404e-08,-1.861594e-08,7.681713e-09
4,1.0,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,...,1.461811e-08,8.484485e-08,1.237115e-07,1.803825e-07,2.630140e-07,1.046957e-06,1.526558e-06,2.225859e-06,3.245505e-06,4.732239e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,1.0,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,...,3.489864e-10,-7.867445e-08,-1.817638e-08,-4.199342e-09,-9.701860e-10,9.466870e-07,2.187158e-07,5.053054e-08,1.167422e-08,2.697128e-09
438,1.0,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,...,2.329427e-06,-2.039047e-07,5.007702e-07,-1.229843e-06,3.020376e-06,1.076534e-07,-2.643865e-07,6.493076e-07,-1.594637e-06,3.916274e-06
439,1.0,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,...,2.945681e-08,1.141706e-06,-3.772376e-07,1.246453e-07,-4.118477e-08,4.831081e-06,-1.596265e-06,5.274313e-07,-1.742717e-07,5.758211e-08
440,1.0,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,...,4.743077e-07,2.345000e-06,-1.365560e-06,7.952041e-07,-4.630698e-07,3.931526e-06,-2.289439e-06,1.333205e-06,-7.763631e-07,4.520982e-07


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y,
                                                    test_size=0.25,
                                                    random_state=42)
lr_poly = LinearRegression()

# Always fit on the training set
lr_poly.fit(X_train, y_train)

lr_poly.score(X_train, y_train)

1.0

In [20]:
lr_poly.score(X_test, y_test)

-29.414739276877288

In [21]:
len(lr_poly.coef_)

1001

In [22]:
mean_squared_error(y_train, lr_poly.predict(X_train), squared=False)

3.9232813487495416e-11

In [23]:
mean_squared_error(y_test, lr_poly.predict(X_test), squared=False)

410.1025158241774

## Data Leakage

[This post about scaling and data leakage](https://datascience.stackexchange.com/questions/38395/standardscaler-before-and-after-splitting-data) explains that if you are going to scale your data, you should only train your scaler on the training data to prevent data leakage.  

We have encountered the idea of splitting our data into two, *training* our model on one bit and then *testing* it on the other.

The goal is to have an unbiased assessment of our model, and so we want to make sure that nothing about our test data sneaks into the training run of the model.

### A Mistake

Now consider the following workflow:

In [24]:
df = pd.concat([pd.DataFrame(data.data, columns=data.feature_names),
               pd.Series(data.target, name='target')], axis=1)

df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [25]:
X, y = load_diabetes(return_X_y=True)

In [26]:
X

array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
         0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
        -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
         0.00286131, -0.02593034],
       ...,
       [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
        -0.04688253,  0.01549073],
       [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
         0.04452873, -0.02593034],
       [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
        -0.00422151,  0.00306441]])

In [27]:
y

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
        68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
        87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
       259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
       128.,  52.,  37., 170., 170.,  61., 144.,  52., 128.,  71., 163.,
       150.,  97., 160., 178.,  48., 270., 202., 111.,  85.,  42., 170.,
       200., 252., 113., 143.,  51.,  52., 210.,  65., 141.,  55., 134.,
        42., 111.,  98., 164.,  48.,  96.,  90., 162., 150., 279.,  92.,
        83., 128., 102., 302., 198.,  95.,  53., 134., 144., 232.,  81.,
       104.,  59., 246., 297., 258., 229., 275., 281., 179., 200., 200.,
       173., 180.,  84., 121., 161.,  99., 109., 115., 268., 274., 158.,
       107.,  83., 103., 272.,  85., 280., 336., 281., 118., 317., 235.,
        60., 174., 259., 178., 128.,  96., 126., 28

In [28]:
ss = StandardScaler().fit(X)
X_scld = ss.transform(X)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_scld, y, random_state=42)

In [30]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.coef_, lr.intercept_)

[  2.27122069 -11.51033366  25.30328309  18.14909453 -43.68872789
  24.17535737   5.56275205  12.81844165  33.09621706   1.25213268] 151.66517559199326


Well we fit the model only to our training data. Looks like we've done everything right, right?

It's important to understand that the answer here is a resounding "NO". It's true that we didn't directly fit our model to our test data. But the trouble is that we fit our scaler **to the whole dataset**. That is, records in the test data are contributing to calculations of column means and standard deviations, and so, surreptitiously, information about our test set is sneaking into the training run of the model after all!

To correct our mistake, we'll make sure to perform our train-test split **first**:

In [31]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=42)

In [32]:
ss2 = StandardScaler().fit(X_train2)
X_train2_scld = ss2.transform(X_train2)
X_test2_scld = ss2.transform(X_test2)

In [33]:
lr2 = LinearRegression()
lr2.fit(X_train2_scld, y_train2)
print(lr2.coef_, lr2.intercept_)

[  2.21507746 -11.51448208  25.07696866  18.24929273 -44.14464184
  24.51385277   5.49759327  13.00675611  33.37981899   1.2479725 ] 154.34441087613294


Note that our model coefficients are slightly different from what they were before.

#### Error Comparison

It's worth pointing out that, **for linear models**, there is **no** difference in modeling error:

In [34]:
y_test_hat = lr.predict(X_test)
mse = mean_squared_error(y_test, y_test_hat)
print(f"Our test RMSE for this model is {round(np.sqrt(mse), 2)}.")

Our test RMSE for this model is 53.37.


In [35]:
y_test2_hat = lr2.predict(X_test2_scld)
mse = mean_squared_error(y_test2, y_test2_hat)
print(f"Our test RMSE for this model is {round(np.sqrt(mse), 2)}.")

Our test RMSE for this model is 53.37.


This will **not** be true for other sorts of models that use different loss functions.

### Preprocessing

In general all preprocessing steps are subject to the same dangers here. Consider the preprocessing step of one-hot-encoding:

In [36]:
gun_poll = pd.read_csv('data/guns-polls.csv')

gun_poll.head()

,Question,Start,End,Pollster,Population,Support,Republican Support,Democratic Support,URL
0,age-21,2/20/18,2/23/18,CNN/SSRS,Registered Voters,72,61,86,http://cdn.cnn.com/cnn/2018/images/02/25/rel3a...
1,age-21,2/27/18,2/28/18,NPR/Ipsos,Adults,82,72,92,https://www.ipsos.com/en-us/npripsos-poll-majo...
2,age-21,3/1/18,3/4/18,Rasmussen,Adults,67,59,76,http://www.rasmussenreports.com/public_content...
3,age-21,2/22/18,2/26/18,Harris Interactive,Registered Voters,84,77,92,http://thehill.com/opinion/civil-rights/375993...
4,age-21,3/3/18,3/5/18,Quinnipiac,Registered Voters,78,63,93,https://poll.qu.edu/national/release-detail?Re...


In [37]:
gun_poll['Pollster'].value_counts()

Pollster
YouGov                 12
Morning Consult        11
Quinnipiac              8
NPR/Ipsos               7
CNN/SSRS                5
CBS News                4
Rasmussen               2
Suffolk                 2
Harris Interactive      1
YouGov/Huffpost         1
SurveyMonkey            1
ABC/Washington Post     1
Harvard/Harris          1
Marist                  1
Name: count, dtype: int64

Now if I were to fit a one-hot encoder to the whole `Pollster` column here, the encoder would learn all the categories. But I need to prepare myself for the real-world possibility that unfamiliar categories may show up in future records. Let's explore this.

In [38]:
# First I'll do a split
X_train, X_test = train_test_split(gun_poll, random_state=42)

In [39]:
X_train['Pollster'].value_counts()

Pollster
Morning Consult        9
YouGov                 9
Quinnipiac             6
NPR/Ipsos              6
CNN/SSRS               3
CBS News               2
Rasmussen              2
Suffolk                2
ABC/Washington Post    1
Marist                 1
YouGov/Huffpost        1
Name: count, dtype: int64

In [57]:
X_train

,Question,Start,End,Pollster,Population,Support,Republican Support,Democratic Support,URL
17,background-checks,2/20/18,2/23/18,CBS News,Adults,75,66,86,https://drive.google.com/file/d/0ByVu4fDHYJgVb...
6,age-21,3/1/18,3/5/18,Morning Consult,Registered Voters,76,72,86,https://morningconsult.com/wp-content/uploads/...
4,age-21,3/3/18,3/5/18,Quinnipiac,Registered Voters,78,63,93,https://poll.qu.edu/national/release-detail?Re...
40,mental-health-own-gun,2/25/18,2/27/18,YouGov,Registered Voters,84,81,86,https://d25d2506sfb94s.cloudfront.net/cumulus_...
19,background-checks,3/1/18,3/5/18,Morning Consult,Registered Voters,90,91,91,https://morningconsult.com/wp-content/uploads/...
36,ban-high-capacity-magazines,2/27/18,2/28/18,NPR/Ipsos,Adults,73,59,88,https://www.ipsos.com/en-us/npripsos-poll-majo...
48,stricter-gun-laws,2/20/18,2/23/18,CNN/SSRS,Registered Voters,70,49,93,http://cdn.cnn.com/cnn/2018/images/02/25/rel3a...
37,ban-high-capacity-magazines,3/3/18,3/5/18,Quinnipiac,Registered Voters,63,39,87,https://poll.qu.edu/national/release-detail?Re...
53,stricter-gun-laws,2/22/18,2/26/18,Morning Consult,Registered Voters,68,53,89,https://morningconsult.com/wp-content/uploads/...
15,background-checks,2/25/18,2/27/18,YouGov,Registered Voters,83,76,91,https://d25d2506sfb94s.cloudfront.net/cumulus_...


In [40]:
X_test['Pollster'].value_counts()

Pollster
YouGov                3
CNN/SSRS              2
Quinnipiac            2
Morning Consult       2
CBS News              2
Harvard/Harris        1
SurveyMonkey          1
Harris Interactive    1
NPR/Ipsos             1
Name: count, dtype: int64

Let's suppose now that I fit a `OneHotEncoder` to the `Pollster` column in my training data.

#### Exercise

Fit an encoder to the `Pollster` column of the training data and then check to see which categories are represented.

In [42]:
dummy_col = X_train[['Pollster']]
ohe = OneHotEncoder(sparse=False)
ohe.fit(dummy_col)

ohe.get_feature_names_out()

/Users/nathan/opt/anaconda3/envs/learn-env-test/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array(['Pollster_ABC/Washington Post', 'Pollster_CBS News',
       'Pollster_CNN/SSRS', 'Pollster_Marist', 'Pollster_Morning Consult',
       'Pollster_NPR/Ipsos', 'Pollster_Quinnipiac', 'Pollster_Rasmussen',
       'Pollster_Suffolk', 'Pollster_YouGov', 'Pollster_YouGov/Huffpost'],
      dtype=object)

<details>
    <summary>Answer</summary>
<code>to_be_dummied = X_train[['Pollster']]
ohe = OneHotEncoder()
ohe.fit(to_be_dummied)
## So what categories do we have?
ohe.get_feature_names()</code>
</details>

We'll want to transform both train and test after we've fitted the encoder to the train.

In [43]:
ohe.transform(dummy_col)

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.

In [47]:
ohe.transform(X_test[['Pollster']])

ValueError: Found unknown categories ['Harris Interactive', 'SurveyMonkey', 'Harvard/Harris'] in column 0 during transform

There are categories in the testing data that don't appear in the training data! What should 
we do about that?

### Approaches

- **Strategy 1**: Divide up the categories proportionally when we do our train_test_split. If we're using `sklearn`'s tool, that means taking advantage of the `stratify` parameter:

In [45]:
new_X_train, new_X_test = train_test_split(gun_poll, 
                                           stratify=gun_poll['Pollster'], random_state=42)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

Unfortunately, in this case, we can't use this since some categories have only a single member.

- **Strategy 2**: Drop the categories with very few representatives.

In the present case, let's try dropping the single-member categories.

In [48]:
vc = gun_poll['Pollster'].value_counts()
to_drop = list(vc[vc == 1].index)
to_drop

['Harris Interactive',
 'YouGov/Huffpost',
 'SurveyMonkey',
 'ABC/Washington Post',
 'Harvard/Harris',
 'Marist']

In [49]:
subset = gun_poll.loc[~gun_poll['Pollster'].isin(to_drop)]

In [50]:
subset['Pollster'].value_counts()

Pollster
YouGov             12
Morning Consult    11
Quinnipiac          8
NPR/Ipsos           7
CNN/SSRS            5
CBS News            4
Rasmussen           2
Suffolk             2
Name: count, dtype: int64

In [51]:
vc_only_1 = vc[vc==1]
bad_cols = vc_only_1.index
bad_cols

Index(['Harris Interactive', 'YouGov/Huffpost', 'SurveyMonkey',
       'ABC/Washington Post', 'Harvard/Harris', 'Marist'],
      dtype='object', name='Pollster')

In [52]:
gun_poll['Pollster'] = gun_poll['Pollster'].map(lambda x: np.nan if x in bad_cols else x)
gun_poll = gun_poll.dropna()

gun_poll['Pollster'].value_counts()

Pollster
YouGov             12
Morning Consult    11
Quinnipiac          8
NPR/Ipsos           7
CNN/SSRS            5
CBS News            4
Rasmussen           2
Suffolk             2
Name: count, dtype: int64

We could now split this carefully so that new categories don't show up in the testing data. In fact, now we can try the stratified split:

In [53]:
X_train3, X_test3 = train_test_split(gun_poll,
                                     stratify=gun_poll['Pollster'],
                                     test_size=0.3,
                                     random_state=42)

X_train3['Pollster'].value_counts()

Pollster
Morning Consult    8
YouGov             8
Quinnipiac         6
NPR/Ipsos          5
CNN/SSRS           3
CBS News           3
Rasmussen          1
Suffolk            1
Name: count, dtype: int64

In [54]:
X_test3['Pollster'].value_counts()

Pollster
YouGov             4
Morning Consult    3
CNN/SSRS           2
Quinnipiac         2
NPR/Ipsos          2
CBS News           1
Rasmussen          1
Suffolk            1
Name: count, dtype: int64

Now every category that appears in the test data appears also in the training data.

- **Strategy 3**: Adjust the settings on the one-hot-encoder.

For `sklearn`'s tool, we'll tweak the `handle_unknown` parameter:

In [55]:
ohe2 = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe2.fit(dummy_col)
ohe2.transform(dummy_col)

/Users/nathan/opt/anaconda3/envs/learn-env-test/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.

In [56]:
ohe2.transform(X_test[['Pollster']])

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

#### Exericse

Fit a new encoder to our training data column that won't break when we try to use it to transform the test data. And then use the encoder to transform both train and test.

<details>
    <summary>Answer</summary>
<code>ohe2 = OneHotEncoder(handle_unknown='ignore')
ohe2.fit(to_be_dummied)
test_to_be_dummied = X_test[['Pollster']]
ohe2.transform(to_be_dummied)
ohe2.transform(test_to_be_dummied)</code>
</details>

## Level Up: $k$-Fold Cross-Validation: Even More Rigorous Validation  

Our goal of using a test set is to simulate what happens when our model attempts predictions on data it's never seen before. But it's possible that our model would *by chance* perform well on the test set.

This is where we could use a more rigorous validation method and turn to **$k$-fold cross-validation**.

![kfolds](images/k_folds.png)

[image via sklearn](https://scikit-learn.org/stable/modules/cross_validation.html)

In this process, we split the dataset into a train set and holdout test sets like usual by performing a shuffling train-test split on the train set.  

We then do $k$-number of _folds_ of the training data. This means we divide the training set into different sections or folds. We then take turns on using each fold as a **validation set** (or **dev set**) and train on the larger fraction. Then we calculate a validation score from the validation set the model has never seen. We repeat this process until each fold has served as a validation set.

This process allows us to try out training our model and check to see if it is likely to overfit or underfit without touching the holdout test data set.

If we think the model is looking good according to our cross-validation using the training data, we retrain the model using all of the training data. Then we can do one final evaluation using the test data. 

It's important that we hold onto our test data until the end and refrain from making adjustments to the model based on the test results.

In [58]:
## Example

X = df.drop('target', axis=1)
y = df['target']

In [59]:
# Let's create our holdout test
X_train, X_test, y_train, y_test = train_test_split(
                                                X,
                                                y,
                                                test_size=0.2,
                                                random_state=42
)

In [62]:
### Simple Model

model_simple = LinearRegression()
scores_simple = cross_validate(model_simple, X_train, y_train, cv=5, return_train_score=True)
print(f"""train scores: {scores_simple['train_score']},
      test scores: {scores_simple['test_score']}""")

train scores: [0.51720249 0.55184827 0.52974211 0.48653503 0.56997653],
      test scores: [0.54760189 0.36123934 0.50481226 0.61773622 0.21489265]


In [64]:
scores_simple

{'fit_time': array([0.03373885, 0.00145793, 0.00300527, 0.00389218, 0.00296426]),
 'score_time': array([0.00781918, 0.00117707, 0.00152612, 0.00173593, 0.00081086]),
 'test_score': array([0.54760189, 0.36123934, 0.50481226, 0.61773622, 0.21489265]),
 'train_score': array([0.51720249, 0.55184827, 0.52974211, 0.48653503, 0.56997653])}

In [63]:
# Mean train r_2
scores_simple['train_score'].mean()

0.5310608857320919

In [66]:
# Mean validation r_2
scores_simple['test_score'].mean()

0.44925647412212166

In [67]:
# Fit on all the training data
model_simple.fit(X_train, y_train)
model_simple.score(X_train, y_train)

0.5279193863361498

### More Complex Model

In [ ]:
# Test out our polynomial model
poly_3 = PolynomialFeatures(3)
X_poly3 = poly_3.fit_transform(X_train)

model_poly3 = LinearRegression()
scores_complex3 = cross_validate(
                        model_poly3, X_poly3, y_train, cv=5, 
                        return_train_score=True
)
print(f"""train scores: {scores_complex3['train_score']},
      test scores: {scores_complex3['test_score']}""")

In [ ]:
# Mean train r_2
np.mean(scores_complex3['train_score']), np.std(scores_complex3['train_score']) 

In [ ]:
# Mean test r_2
np.mean(scores_complex3['test_score']), np.std(scores_complex3['test_score'])

In [ ]:
# Fit on all the training data
model_poly3.fit(X_poly3, y_train)
model_poly3.score(X_poly3, y_train)

### Medium-Complexity Model

In [ ]:
# Test out our polynomial model
poly_2 = PolynomialFeatures(2)
X_poly2 = poly_2.fit_transform(X_train)

model_poly2 = LinearRegression()
scores_complex2 = cross_validate(
                        model_poly2, X_poly2, y_train, cv=5, 
                        return_train_score=True
)
print(f"""train scores: {scores_complex2['train_score']},
      test scores: {scores_complex2['test_score']}""")

In [ ]:
# Mean train r_2
np.mean(scores_complex2['train_score']), np.std(scores_complex2['train_score']) 

In [ ]:
# Mean test r_2
np.mean(scores_complex2['test_score']), np.std(scores_complex2['test_score'])

model_poly2.fit(X_poly2, y_train)
model_poly2.score(X_poly2, y_train)

### Checking Our Models Against the Holdout Test Set

Once we have an acceptable model, we train our model on the entire training set and score on the test to validate.

In [ ]:
best_model = None

In [ ]:
# Remember that we have to transform X_test in the same way
None

## Leakage into Validation Data

If we employ cross-validation, then our training data points will be serving both for training and for validation. So there's a sense in which we can't help but let some information about our validation data sneak into the model.

But strictly speaking, cross-validation means building *multiple* models, and we still want each to be blind to its validation set.

The dangers of data leakage, therefore, are still very much real in the case of validation data. And they are often more subtle as well. Consider the following line of code:

In [ ]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

# Using our scaled training data

cv_results = cross_validate(estimator=LinearRegression(),
                X=X_train2_scld,
                y=y_train2,
                scoring=mse,
                return_estimator=True)

We've built five models here, and none of them saw any points from the test data, so we have no leaks, right?

Wrong! We fit the `StandardScaler` to the whole training set, which means that information about *every* fold will affect every cross-validation. A better practice here would be to split our data into its cross-validation folds *first*. Then we can fit the scaler to only the training folds for each cross-validation.

Of course, the more preprocessing steps we have, the more tedious it becomes to do this work! For such tasks it is often greatly beneficial to take advantage of `sklearn`'s `Pipeline`s, which we'll have more to say about later.

For now, let's see if we can fix our leaky cross-validation scorer.

#### Exercise

Go back to `X_train2` and do the following:

- Split it into five validation folds (Use `KFold()`).
- For each split:
- (i) fit a `StandardScaler` to the four-fold chunk and transform all data points with it.
- (ii) fit a `LinearRegression` to the four-fold chunk and print out the value of the first coefficient.

<details>
    <summary>Answer</summary>

<code>for train_ind, val_ind in KFold().split(X_train2):
    train = X_train2[train_ind, :]
    val = X_train2[val_ind, :]
    target_train = y_train2[train_ind]
    target_val = y_train2[val_ind]
    ss = StandardScaler().fit(train)
    train_scld = ss.transform(train)
    val_scld = ss.transform(val)
    lr = LinearRegression().fit(train_scld, target_train)
    print(lr.coef_[0])</code>
</details>